In [2]:
import rpy2
%load_ext rpy2.ipython

import numpy as np
from glmnet import GLMNet
from glmnet import RegGLM
rng = np.random.default_rng(0)
import statsmodels.api as sm

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [3]:
i_, s_ = True, False
n, p = 100, 50
X = rng.standard_normal((n, p))
beta = np.zeros(p)
beta[:2] = [1,2]
Rfam, fam = 'binomial', sm.families.Binomial()
y = (rng.standard_normal(n) + X @ beta > 0).astype(int)
#y = rng.standard_normal(n) + X @ beta

W = rng.uniform(1, 2, size=(n,))
W *= n / W.sum()

In [4]:
%%R -i X,W,y,n,p,s_,i_,Rfam -o coef_,intercept_,L
y = as.numeric(y)
library(glmnet)
G = glmnet(X, y, standardize=s_, intercept=i_, family=Rfam, weights=as.numeric(W), alpha=0.5)
B = predict(G, type='coef')
L = G$lambda 
print(max(G$lambda)*n)
coef_ = as.matrix(B[2:(p+1),])
intercept_ = as.numeric(B[1,])


[1] 73.84822


Loading required package: Matrix
Loaded glmnet 4.1-7


In [23]:
G = GLMNet(lambda_values=L,
           lambda_fractional=False,
           standardize=s_, 
           fit_intercept=i_, 
           family=fam, 
           alpha=0.5)
G.fit(X, y, sample_weight=W)
G.lambda_max_, L.max()

(0.7384822402986349, 0.7384822402986349)

In [24]:
np.linalg.norm(coef_.T - G.coefs_) / np.linalg.norm(coef_)

0.003313554032755549

In [25]:
coef_.shape, G.coefs_.shape

((50, 100), (100, 50))

In [18]:
idx = 15
G.coefs_[idx]

array([0.08115015, 0.83036112, 0.        , 0.        , 0.00463349,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [29]:
G_ = RegGLM(lambda_val=L[idx],
            alpha=0.5,
            standardize=s_, 
            fit_intercept=i_, 
            family=fam).fit(X, y, sample_weight=W)
G_.coef_

array([0.08115111, 0.83036152, 0.        , 0.        , 0.0046335 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [22]:
coef_[:,idx]

array([0.08115134, 0.83036158, 0.        , 0.        , 0.0046335 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])